In [1]:
!pip3 install playwright
!pip3 install selectolax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 37.6 MB/s eta 0:00:00


In [2]:
!playwright install chromium

163.5 MiB [] 0% 0.0s163.5 MiB [] 0% 73.7s163.5 MiB [] 0% 84.2s163.5 MiB [] 0% 25.7s163.5 MiB [] 0% 18.1s163.5 MiB [] 0% 15.6s163.5 MiB [] 0% 15.1s163.5 MiB [] 0% 14.7s163.5 MiB [] 1% 13.8s163.5 MiB [] 1% 13.1s163.5 MiB [] 1% 12.0s163.5 MiB [] 1% 11.5s163.5 MiB [] 1% 11.2s163.5 MiB [] 2% 10.5s163.5 MiB [] 2% 10.1s163.5 MiB [] 2% 9.9s163.5 MiB [] 2% 9.6s163.5 MiB [] 3% 9.4s163.5 MiB [] 3% 9.5s163.5 MiB [] 3% 9.1s163.5 MiB [] 3% 9.0s163.5 MiB [] 4% 8.8s163.5 MiB [] 4% 8.9s163.5 MiB [] 4% 9.0s163.5 MiB [] 4% 9.3s163.5 MiB [] 4% 9.1s163.5 MiB [] 5% 8.9s163.5 MiB [] 5% 8.8s163.5 MiB [] 5% 8.4s163.5 MiB [] 6% 7.9s163.5 MiB [] 6% 8.3s163.5 MiB [] 6% 8.1s163.5 MiB [] 7% 7.9s163.5 MiB [] 7% 7.6s163.5 MiB [] 7% 7.4s163.5 MiB [] 8% 7.1s163.5 MiB [] 8% 7.0s163.5 MiB [] 9% 6.9s163.5 MiB [] 9% 7.0s163.5 MiB [] 9% 6.9s163.5 MiB [] 10% 6.8s163.5 MiB [] 10% 6.7s163.5 MiB [] 10% 6.8s163.5 MiB [] 10% 6.9s163.5 MiB [] 10% 6.8s163.5 MiB [] 11% 6.9s163.5 MiB [] 11% 6.7s163.5 MiB [] 12% 6.5s163.5 MiB [] 12% 6

In [3]:
url = "https://store.steampowered.com/specials" # Javascript-heavy website

In [4]:
# import asyncio
# from playwright.async_api import async_playwright

# async with async_playwright() as p:
#     browser = await p.chromium.launch(headless=True)
#     page = await browser.new_page()
#     await page.goto(url)
#     # await page.screenshot(path="screenshot.png", full_page=True) # From screenshot, we can see not everything is loaded in the webpage so our parsing strategy will fail.

#     TIMEOUT = 300001
#     await page.wait_for_load_state('networkidle', timeout=TIMEOUT) # When there is nothing left to render and network is literally idle
#     await page.evaluate("() => window.scroll(0, document.body.scrollHeight)") # Sometimes, for performance, websites renders information as user scrolls down to the page. So, to get full page content, we have to imitate scroll down on the website
#     await page.screenshot(path="screenshot2.png", full_page=True) # From screenshot2, we can see everything is loaded now.
#     await browser.close()

In [8]:
import asyncio
from playwright.async_api import async_playwright
from selectolax.parser import HTMLParser

async def extract_full_body_html(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(url, wait_until="domcontentloaded")

        # Repeatedly scroll until we reach the bottom.
        previous_height = 0
        while True:
            # Scroll to the bottom
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")

            await page.wait_for_load_state('domcontentloaded')

            # await page.wait_for_selector('div[class*="gASJ2lL_xmVNuZkWGvrWg"]')

            # Wait briefly for any new content to load
            await page.wait_for_timeout(2000)

            # Check the new scroll height
            new_height = await page.evaluate("document.body.scrollHeight")
            if new_height == previous_height:
                # No new content was loaded, so we’re done
                break
            previous_height = new_height

        html = await page.inner_html("body")

        await asyncio.sleep(2)

        # Now that we've scrolled fully, take the screenshot
        await page.screenshot(path="full_screenshot.png", full_page=True)

        await browser.close()

        return html


[<Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>] 12


In [ ]:
def get_divs(html):
    tree = HTMLParser(html)
    divs = tree.css('div[class*="gASJ2lL_xmVNuZkWGvrWg"]') # This class name keeps changing. That is a problem
    return divs

In [9]:
# # Running inside Jupyter Notebook
# async def main():
#     html = await extract_full_body_html(url)
#     print(get_divs(html), len(get_divs(html)))

# # Run the async function properly
# await main()
html = await extract_full_body_html(url)

[<Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>, <Node div>] 12


In [10]:
# print(get_divs(html), len(get_divs(html)))
divs = get_divs(html)

In [31]:
for d in divs:
    title = d.css_first('div[class*="StoreSaleWidgetTitle"]').text(strip=True)
    thumbnail = d.css_first('div[class*="CapsuleImageCtn"] > img').attributes.get('src')
    tags = [tag.text() for tag in d.css('div[class*="gASJ2lL_xmVNuZkWGvrWg"] a[class*="WidgetTag"]')[:5]]
    review = d.css_first('div[class*="gASJ2lL_xmVNuZkWGvrWg"] a[class*="ReviewScore"] > div > div')

    release_date_div = review.previous_sibling
    release_date = d.css(f'{release_date_div} > div').text(strip=True)

    attrs = {
        'title': title,
        'tags': tags,
        'thumbnail': thumbnail,
        'review': review.text(strip=True) if review else None,
        'release_date': release_date_div
    }
    print(attrs)

AttributeError: 'selectolax.parser.Node' object has no attribute 'previous_sibling'